# Test notebook for tpot model generation -> onnx

In [ ]:
import logging
import log

log.setup()

LOGGER = logging.getLogger('test.model2onnx')
LOGGER.info("logger ready")

RANDOM_SEED = 0

AUTOML_PARAMS = {
    'skautoml': {
        'per_run_time_limit': 60, # 120,
        'max_train_time': 60, # 900,
        'n_jobs': 4,
        'random_state': RANDOM_SEED,
    },
    'tpot': {
        'population_size': 100,
        'n_jobs': 4,
        'verbosity': 2,
        'max_train_time': 60, # 1200,
        'generations': 100,
        'early_stop': 15,
        'template': 'Selector-Transformer-Regressor',
        'random_state': RANDOM_SEED,
    }
}

In [ ]:
from fantasy_py.lineup.strategy import Contest
from fantasy_py.fantasy_types import ContestStyle

from generate_train_test import generate_train_test, load_csv
from automl import create_automl_model


def generate_model(sport, service, style, contest_type, target, model_type):
    print(f"{sport=} {service=} {contest_type=} {style=} {target=}")
    df = load_csv(sport, service, style, contest_type, data_folder="data")

    model_data = generate_train_test(
        df,
        random_state=0,
    )
    (X_train, X_test, y_top_train, y_top_test,
        y_last_win_train, y_last_win_test) = model_data
    y_train, y_test = (
        (y_top_train, y_top_test) 
        if target.startswith("top") else 
        (y_last_win_train, y_last_win_test)
    )
    ml_params = dict(AUTOML_PARAMS[model_type])
    cam_result = create_automl_model(
        target,
        framework=model_type,
        X_train=X_train, y_train=y_train,
        X_test=X_test, y_test=y_test,
        **ml_params
    )
    return cam_result, X_train, y_train


In [ ]:
target = "top-score"
model_type = 'tpot'
output_dir = "eval_results"
model_result, X_train, y_train = generate_model("nfl", "fanduel", "CLASSIC", "FIFTY_FIFTY", target, 'tpot')

In [ ]:
from serialize import serialize_model

serialize_ex = None
try:
    onnx_model = serialize_model(
        model_result['model'], model_type,
        X_train, y_train,
        model_type + "-" + target,
        model_folder=output_dir,
        model_desc_folder=output_dir,
    )
except Exception as ex:
    import traceback
    traceback.print_exc(limit=None, chain=True)
    serialize_ex = ex
